<a href="https://colab.research.google.com/github/dacenish/Travel-Package-Prize-Prediction-Model-Using-Flask/blob/main/REST_API_For_Travel_Package_Prize_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install flask

In [51]:
!pip install flask_ngrok pyngrok

In [52]:
!pip install sentence-transformers

In [53]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
from pyngrok import ngrok  # import ngrok
from datetime import datetime
from google.colab import drive
from flask_ngrok import run_with_ngrok  # fix the import for run_with_ngrok
from sklearn.preprocessing import StandardScaler
from flask import Flask, render_template, request, jsonify

In [54]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sentence_transformers import SentenceTransformer
import pandas as pd
import pickle


model_file_path = '/content/drive/MyDrive/models/decision_tree_reg_latest.pkl'

# load the model from the file
with open(model_file_path, 'rb') as model_file:
  loaded_decision_tree_reg = pickle.load(model_file)

import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Defining the PCA and StandardScaler object

pca = PCA(n_components=23)
scaler = StandardScaler()

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v4_MiniLM-L6')

In [56]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Initialize the SentenceTransformer model
model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v4_MiniLM-L6')

# Create a function for prediction
def predict_price(input_data, lr_model, pca, scaler):
    # Prepare the input data
    text_columns = [
        'Package Name', 'Destination', 'Itinerary', 'Places Covered',
        'Hotel Details', 'Airline', 'Sightseeing Places Covered', 'Cancellation Rules'
    ]

    # Initialize an empty DataFrame
    df = pd.DataFrame([input_data])

    # Encode text-based columns and create embeddings
    for column in text_columns:
        df[column + '_embedding'] = df[column].apply(lambda text: model.encode(text))

    # Apply PCA separately to each text embedding column
    n_components = 23  # Adjust the number of components as needed
    text_embeddings_pca = np.empty((len(df), n_components * len(text_columns)))

    for i, column in enumerate(text_columns):
        embeddings = df[column + '_embedding'].values.tolist()
        embeddings_pca = pca.transform(embeddings)
        text_embeddings_pca[:, i * n_components:(i + 1) * n_components] = embeddings_pca

    # Combine text embeddings with other numerical features if available
    numerical_features = [
        'Package Type_Standard', 'Package Type_Premium', 'Package Type_Luxury',
        'Travel_Month', 'Package Type_Budget', 'Package Type_Deluxe',
        'Hotel Ratings', 'Start City_New Delhi', 'Start City_Mumbai',
        'Travel_DayOfWeek', 'Travel_Year'
    ]

    X_numerical = df[numerical_features].values

    # Combine PCA-transformed text embeddings and numerical features
    X = np.hstack((text_embeddings_pca, X_numerical))

    # Scale the data using the same scaler used during training
    X = scaler.transform(X)

    # Make predictions using the trained Linear Regression model
    y_pred = lr_model.predict(X)

    return y_pred[0]

In [57]:
# Loaded the dataset from the previous project after cleaning.
df_input = pd.read_csv('/content/drive/MyDrive/Travel Package Price Prediction/marketing_sample_for_makemytrip_com-travel__20190901_20190930__30k_data.csv',on_bad_lines='skip')
df_input.head()

,Uniq Id,Crawl Timestamp,Package Name,Page Url,Package Type,Company,Destination,Itinerary,Places Covered,Travel Date,...,Airline,Flight Stops,Onwards Return Flight Time,Meals,Price Per Two Persons,Per Person Price,Sightseeing Places Covered,Initial Payment For Booking,Cancellation Rules,Date Change Rules
0,0eaaee0a8a6ba47cfc9f6e6852fcf9f0,2019-09-19 23:07:23 +0000,A Grand Week in North East - Lachung Special,https://holidayz.makemytrip.com/holidays/india...,Deluxe,Makemytrip,Gangtok|Lachung|Gangtok|Darjeeling,2N Gangtok . 2N Lachung . 1N Gangtok . 2N Darj...,Gangtok|Lachung|Gangtok|Darjeeling,02-10-2019,...,IndiGo,NaN,14 : 00-16 : 05,NaN,71350,35675.0,Baba Mandir | MG Road - Walk | Visit to Lake ...,NaN,Cancellation any time after making the 1st pay...,NaN
1,c782c902e6d370153361d708bb2bca64,2019-09-25 16:15:44 +0000,A Relaxing holiday to Kerala - Free Speed Boat...,https://holidayz.makemytrip.com/holidays/india...,Standard,Makemytrip,Cochin|Munnar|Thekkady|Allepey|Kovalam and Poovar,1N Cochin . 2N Munnar . 1N Thekkady . 1N Allep...,Cochin|Munnar|Thekkady|Allepey|Kovalam and Poovar,02-10-2019,...,Spicejet|IndiGo,NaN,15 : 45-17 : 45|10 : 50-13 : 00,NaN,32864,16432.0,Dutch Palace | Jewish Synagogue | St. Francis...,NaN,Cancellation any time after making the 1st pay...,NaN
2,1087ed9303d0ac724460088a5920065d,2019-09-26 04:18:42 +0000,Experiential Manali from Chandigarh (Candid Ph...,https://holidayz.makemytrip.com/holidays/india...,Deluxe,Makemytrip,Manali,3N Manali,Manali,24-10-2019,...,NaN,NaN,NaN,NaN,12045,6022.5,Vashishth Kund | Hadimba Temple | Tibetan Mon...,NaN,This cancellation policy is applicable for boo...,NaN
3,3bda4788836d573910f605154ba949dc,2019-09-24 11:46:20 +0000,Stunning Kashmir Holiday,https://holidayz.makemytrip.com/holidays/india...,Standard,Makemytrip,Gulmarg|Pahalgam|Srinagar,1N Gulmarg . 2N Pahalgam . 2N Srinagar,Gulmarg|Pahalgam|Srinagar,26-10-2019,...,IndiGo|IndiGo,NaN,06 : 30-09 : 10|12 : 45-14 : 20,NaN,31933,15966.5,Gondola Point | Avantipura Ruins | Mughal Gar...,NaN,Cancellation any time after making the 1st pay...,NaN
4,aab612682c998bde97e040b212cd5689,2019-09-25 12:45:50 +0000,Best of Holiday in Himachal from Chandigarh,https://holidayz.makemytrip.com/holidays/india...,Standard,Makemytrip,Shimla|Manali|Dharamshala|Dalhousie|Chandigarh,2N Shimla . 3N Manali . 2N Dharamshala . 2N Da...,Shimla|Manali|Dharamshala|Dalhousie|Chandigarh,13-10-2019,...,NaN,NaN,NaN,NaN,35725,17862.5,Pinjore Gardens | Mall road | Kufri | Indira ...,NaN,Cancellation any time after making the 1st pay...,NaN


In [58]:

# Filling missing values for Hotel Details with 'Not Available'
df_input['Hotel Details'].fillna('Not Available', inplace=True)

# Filling missing values for Airline with 'Not Available'
df_input['Airline'].fillna('Not Available', inplace=True)

# Filling missing values for Onwards Return Flight Time with 'Not Available'
df_input['Onwards Return Flight Time'].fillna('Not Available', inplace=True)

# Filling missing values for Sightseeing Places Covered with 'Not Available'
df_input['Sightseeing Places Covered'].fillna('Not Available', inplace=True)

# Filling missing values for Initial Payment For Booking with 0 (assuming no initial payment)
df_input['Initial Payment For Booking'].fillna(0, inplace=True)

# Filling missing values for Cancellation Rules with 'Not Available'
df_input['Cancellation Rules'].fillna('Not Available', inplace=True)

# Dropping columns with all missing values (Flight Stops, Date Change Rules, Unnamed: 22, Unnamed: 23)
df_input.drop(columns=["Flight Stops", "Meals", "Initial Payment For Booking", "Date Change Rules"], inplace=True)
df_input['Travel Date'] = pd.to_datetime(df_input['Travel Date'], format='%d-%m-%Y', errors='coerce')
allowed_package_types = ['Deluxe', 'Standard', 'Premium', 'Luxury', 'Budget']

# Filter the DataFrame to keep only the rows with allowed package types
df_input = df_input[df_input['Package Type'].isin(allowed_package_types)]
df_input.drop('Company', axis=1, inplace=True)
df_input.drop('Crawl Timestamp', axis=1, inplace=True)

# Extracting hotel ratings from Hotel Details
df_input['Hotel Ratings'] = df_input['Hotel Details'].str.extract(r'(\d+\.\d+)')

# Convert 'Hotel Ratings' to numeric (float)
df_input['Hotel Ratings'] = pd.to_numeric(df_input['Hotel Ratings'], errors='coerce')

# Calculate the mode of the 'Hotel Ratings' column
mode_rating = df_input['Hotel Ratings'].mode()[0]

# Replace NaN values with the mode
df_input['Hotel Ratings'].fillna(mode_rating, inplace=True)

# Assuming your data is stored in a DataFrame called 'df_input'
df_input['Travel Date'] = pd.to_datetime(df_input['Travel Date'])
df_input['Travel_Year'] = df_input['Travel Date'].dt.year
df_input['Travel_Month'] = df_input['Travel Date'].dt.month
df_input['Travel_DayOfWeek'] = df_input['Travel Date'].dt.dayofweek

# Example of one-hot encoding
df_input = pd.get_dummies(df_input, columns=['Package Type','Start City'])

Q1 = df_input['Per Person Price'].quantile(0.25)
Q3 = df_input['Per Person Price'].quantile(0.75)
IQR = Q3 - Q1

# Step 2: Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 3: Filter the DataFrame to remove outliers
df_input = df_input[(df_input['Per Person Price'] >= lower_bound) & (df_input['Per Person Price'] <= upper_bound)]


In [59]:
row_index = 100  # Replace with the desired row index

# Access the row from df_input using iloc
input_row = df_input.iloc[row_index]

# Create an input dictionary from the selected row
input_data = {
    'Package Name': input_row['Package Name'],
    'Destination': input_row['Destination'],
    'Itinerary': input_row['Itinerary'],
    'Places Covered': input_row['Places Covered'],
    'Hotel Details': input_row['Hotel Details'],
    'Airline': input_row['Airline'],
    'Sightseeing Places Covered': input_row['Sightseeing Places Covered'],
    'Cancellation Rules': input_row['Cancellation Rules'],
    'Package Type_Standard': input_row['Package Type_Standard'],
    'Package Type_Premium': input_row['Package Type_Premium'],
    'Package Type_Luxury': input_row['Package Type_Luxury'],
    'Travel_Month': input_row['Travel_Month'],
    'Package Type_Budget': input_row['Package Type_Budget'],
    'Package Type_Deluxe': input_row['Package Type_Deluxe'],
    'Hotel Ratings': input_row['Hotel Ratings'],
    'Start City_New Delhi': input_row['Start City_New Delhi'],
    'Start City_Mumbai': input_row['Start City_Mumbai'],
    'Travel_DayOfWeek': input_row['Travel_DayOfWeek'],
    'Travel_Year': input_row['Travel_Year']
}
import joblib

# Load the models from the files
loaded_lr_model = joblib.load('/content/drive/MyDrive/models/linear_reg_latest.pkl')
loaded_pca = joblib.load('/content/drive/MyDrive/models/pca.joblib')
loaded_scaler = joblib.load('/content/drive/MyDrive/models/scaler.joblib')

# predicted_price = str(predict_price(input_data, loaded_lr_model, loaded_pca, loaded_scaler))[-10:]
# print(f'Predicted Per Person Price: ${predicted_price}')


In [60]:
!ngrok authtoken "2Y4QjYS22cNYNBPIWdi6jnMYh4r_6WzGhpDjNDpzwJubsR8dL"

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/', methods=['GET', 'POST'])
def predict():
    return """

    <!DOCTYPE html>
<html>
<head>
    <title>Travel Package Prediction</title>
    <style>
        body {
            font-family: 'Poppins', sans-serif;
            background-color: #f9f9f9;
            margin: 0;
            padding: 0;
        }

        .container {
            max-width: 800px;
            margin: 0 auto;
            padding: 40px;
            background-color: #ffffff;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.1);
            text-align: center;
        }

        h1 {
            color: #007BFF;
            font-size: 36px;
            margin-bottom: 20px;
        }

        form {
            text-align: left;
        }

        input[type="text"],
        input[type="number"] {
            width: 100%;
            padding: 15px;
            margin: 15px 0;
            border: none;
            border-bottom: 2px solid #007BFF;
            font-size: 18px;
            background-color: transparent;
            color: #333;
            transition: border-bottom 0.3s ease;
        }

        input[type="text"]:focus,
        input[type="number"]:focus {
            border-bottom: 2px solid #0056b3;
            outline: none;
        }

        input[type="checkbox"],
        input[type="radio"] {
            margin-right: 10px;
        }

        input[type="submit"] {
            background-color: #007BFF;
            color: #fff;
            padding: 15px 30px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 20px;
            transition: background-color 0.3s ease;
        }

        input[type="submit"]:hover {
            background-color: #0056b3;
        }

        p#prediction {
            margin-top: 20px;
            font-size: 24px;
            color: #007BFF;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Travel Package Prediction</h1>
        <form action="/predict" method="POST">
            <label for="Package Name">Package Name:</label>
            <input type="text" name="Package Name" placeholder="Custom Package" value="A Grand Week in North East - Lachung Special">

            <label for="Destination">Destination:</label>
            <input type="text" name="Destination" placeholder="Custom Destination" value="Gangtok|Lachung|Gangtok|Darjeeling">

            <label for="Itinerary">Itinerary:</label>
            <input type="text" name="Itinerary" placeholder="Custom Itinerary" value="2N Gangtok . 2N Lachung . 1N Gangtok . 2N Darjeeling">

            <label for="Places Covered">Places Covered:</label>
            <input type="text" name="Places Covered" placeholder="Custom Places" value="Gangtok|Lachung|Gangtok|Darjeeling">

            <label for="Hotel Details">Hotel Details:</label>
            <input type="text" name="Hotel Details" placeholder="Custom Hotel Details" value="Lemon Tree Hotel  Gangtok:4.2|Summit Alpine Resort, Lachung- MMT Holidays Special:4.2|Lemon Tree Hotel  Gangtok:4.4|Ramada by Wyndham Darjeeling Gandhi Road:">

            <label for="Airline">Airline:</label>
            <input type="text" name="Airline" placeholder="Custom Airline" value="Air India|IndiGo">

            <label for="Sightseeing Places Covered">Sightseeing Places Covered:</label>
            <input type="text" name="Sightseeing Places Covered" placeholder="Custom Sightseeing" value="Baba Mandir | MG Road - Walk | Visit to Lake Tsomgo with Yak Safari | Snacks at Changu lake (Winter Special) | Yumthang Valley | Hot Spring">

            <label for="Cancellation Rules">Cancellation Rules:</label>
            <input type="text" name="Cancellation Rules" placeholder="Custom Cancellation Rules" value="Not Available">


            <label for="Package Type_Standard">Package Type_Standard:</label>
            <input type="checkbox" name="Package Type_Standard" value="1" checked>

            <label for="Package Type_Premium">Package Type_Premium:</label>
            <input type="checkbox" name="Package Type_Premium" value="1">

            <label for="Package Type_Luxury">Package Type_Luxury:</label>
            <input type="checkbox" name="Package Type_Luxury" value="1">

            <label for="Travel_Month">Travel Month:</label>
            <input type="number" name="Travel_Month" min="1" max="12" placeholder="Travel Month" value="5">

            <label for="Package Type_Budget">Package Type_Budget:</label>
            <input type="checkbox" name="Package Type_Budget" value="1">

            <label for="Package Type_Deluxe">Package Type_Deluxe:</label>
            <input type="checkbox" name="Package Type_Deluxe" value="1">

            <label for="Hotel Ratings">Hotel Ratings:</label>
            <input type="number" name="Hotel Ratings" placeholder="Hotel Ratings" value="4.5">

            <label for="Start City_New Delhi">Start City_New Delhi:</label>
            <input type="checkbox" name="Start City_New Delhi" value="1">

            <label for="Start City_Mumbai">Start City_Mumbai:</label>
            <input type="checkbox" name="Start City_Mumbai" value="1">

            <label for="Travel_DayOfWeek">Travel Day of Week:</label>
            <input type="number" name="Travel_DayOfWeek" min="0" max="6" placeholder="Travel Day of Week" value="2">

            <label for="Travel_Year">Travel Year:</label>
            <input type="number" name="Travel_Year" min="2023" max="2123" placeholder="Travel Year" value="2023">

            <input type="submit" value="Predict">
        </form>
        <p id="prediction"></p>
    </div>
</body>
</html>


    """


@app.route('/predict', methods=['POST'])
def index():
    if request.method == 'POST':
        # Get input data from the form
        package_name = request.form.get('Package Name')
        destination = request.form.get('Destination')
        itinerary = request.form.get('Itinerary')
        places_covered = request.form.get('Places Covered')
        hotel_details = request.form.get('Hotel Details')
        airline = request.form.get('Airline')
        sightseeing_places = request.form.get('Sightseeing Places Covered')
        cancellation_rules = request.form.get('Cancellation Rules')
        package_standard = int(request.form.get('Package Type_Standard', 0))
        package_premium = int(request.form.get('Package Type_Premium', 0))
        package_luxury = int(request.form.get('Package Type_Luxury', 0))
        travel_month = int(request.form.get('Travel_Month'))
        package_budget = int(request.form.get('Package Type_Budget', 0))
        package_deluxe = int(request.form.get('Package Type_Deluxe', 0))
        hotel_ratings = float(request.form.get('Hotel Ratings'))
        start_city_delhi = int(request.form.get('Start City_New Delhi', 0))
        start_city_mumbai = int(request.form.get('Start City_Mumbai', 0))
        travel_day_of_week = int(request.form.get('Travel_DayOfWeek'))
        travel_year = int(request.form.get('Travel_Year'))

        # Create a dictionary to store the input data
        data = {
            'Package Name': package_name,
            'Destination': destination,
            'Itinerary': itinerary,
            'Places Covered': places_covered,
            'Hotel Details': hotel_details,
            'Airline': airline,
            'Sightseeing Places Covered': sightseeing_places,
            'Cancellation Rules': cancellation_rules,
            'Package Type_Standard': package_standard,
            'Package Type_Premium': package_premium,
            'Package Type_Luxury': package_luxury,
            'Travel_Month': travel_month,
            'Package Type_Budget': package_budget,
            'Package Type_Deluxe': package_deluxe,
            'Hotel Ratings': hotel_ratings,
            'Start City_New Delhi': start_city_delhi,
            'Start City_Mumbai': start_city_mumbai,
            'Travel_DayOfWeek': travel_day_of_week,
            'Travel_Year': travel_year
        }

        # Perform prediction using the custom_input dictionary
        prediction = predict_price(data, loaded_lr_model, loaded_pca, loaded_scaler)//10
        prediction = str(prediction)[-7:]

        return jsonify({'prediction': prediction})


if __name__ == "__main__":
    app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6ff6-34-125-82-134.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Nov/2023 09:49:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2023 09:49:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2023 09:49:04] "POST /predict HTTP/1.1" 200 -
